<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

!pip freeze > requirements.txt

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [67]:
# текст запроса
query_3_1 = f'''
                SELECT COUNT(*)
                FROM vacancies
            '''

In [68]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3515521326.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [69]:
# текст запроса
query_3_2 = f'''
                SELECT COUNT(*)
                FROM employers
            '''

In [70]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3655965175.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [71]:
# текст запроса
query_3_3 = f'''
                SELECT COUNT(*)
                FROM areas
            '''

In [72]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3731397118.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [73]:
# текст запроса
query_3_4 = f'''
                SELECT COUNT(*)
                FROM industries
            '''

In [74]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\619965657.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,count
0,294


***

#### Дополнительные запросы для дополнения выводов по блоку

Топ 5 компаний по числу вакансий.

In [75]:
# текст запроса
query_3_5= f'''
                SELECT  
                    e.name employer,
                    COUNT(v.id)
                FROM vacancies v
                JOIN employers e ON v.employer_id = e.id
                GROUP BY e.id
                ORDER BY COUNT(v.id) DESC
                LIMIT 5
            '''
# результат запроса
df = pd.read_sql_query(query_3_5, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\1961838542.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_5, connection)


,employer,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


Найдем количество работодателей, у которых в данный момент нет размещенных вакансий.

In [76]:
# текст запроса
query_3_6= f'''
            SELECT  
            COUNT(*) null_vacancies_employers
            FROM employers e
            LEFT JOIN  vacancies v ON v.employer_id = e.id
            WHERE v.id IS NULL
            '''
# результат запроса
df = pd.read_sql_query(query_3_6, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\4169263512.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_6, connection)


,null_vacancies_employers
0,8595


Среди работодателей, у которых размещена хотябы одна вакансия, найдем средние число вакансий.

In [77]:
# текст запроса
query_3_7= f'''
            SELECT
            AVG(aa.count) avg_vacancies_on_employer
            FROM (
                SELECT  
                    e.name,
                    count(v.id)
                FROM vacancies v
                JOIN employers e ON v.employer_id = e.id
                GROUP BY E.ID
                ) aa
            '''
# результат запроса
df = pd.read_sql_query(query_3_7, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\2964029651.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_7, connection)


,avg_vacancies_on_employer
0,3.300483


---

#### Выводы по предварительному анализу данных:

* По результатам запросов мы узнали, что количество вакансий, работодателей, регионов и сфер деятельности равно соответственно 49197, 23501, 1362 и 294.

* Больше всего вакансий размещено компанией Яндекс - 1933, в топ 5 по количеству объявлений о вакансии также входят Ростелеком, Тинькофф, СБЕР и Газпром нефть, что объясняется тем, что чем больше компания и темп ее роста, тем больше новых сотрудников ей требуется.

* Интересно отметить, что среди работодателей в нашей базе данных есть 8595, у которых в данный момент не размещено ни одной вакансии.

* Среди работодателей, у которых размещена хотябы одна вакансия, средние число вакансий равно 3.3.



# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [78]:
# текст запроса
# текст запроса
query_4_1= f'''
            SELECT 
                a.name area,
            COUNT(v.id) cnt
            FROM vacancies v 
            JOIN areas a ON a.id = v.area_id
            GROUP BY a.id
            ORDER BY 2 DESC 
            LIMIT 5
            '''


In [79]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\234815181.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [80]:
# текст запроса
query_4_2= f'''
            SELECT 
                COUNT(id) cnt
            FROM vacancies  
            WHERE COALESCE(salary_from, salary_to) IS NOT NULL  /* coalesce вернет нам первый не NULL
            параметр*/
            '''


In [81]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3734613624.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [82]:
# текст запроса
# текст запроса
query_4_3= f'''
            SELECT 
                round(AVG(salary_from)) avg_salary_from,
                round(AVG(salary_to)) avg_salary_to
            FROM vacancies  
            '''


In [83]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\327040603.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [84]:
# текст запроса
query_4_4= f'''
            SELECT 
                schedule || ' - ' || employment combination,
                COUNT(id)
            FROM vacancies 
            GROUP BY schedule, employment
            ORDER BY 2 DESC
            OFFSET 1 LIMIT 1
            '''


In [85]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3274844153.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


,combination,count
0,Удаленная работа - Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [86]:
# текст запроса
query_4_5= f'''
            SELECT 
                experience,
                COUNT(id)
            FROM vacancies 
            GROUP BY experience
            ORDER BY 2 
            '''


In [87]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\969833449.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


---
#### Дополнительные запросы для дополнения выводов по блоку

Выясним дополнительно сколько вакансий приходится на топ 15 городов

In [88]:
# текст запроса

query_4_6= f'''
            WITH x AS(
                SELECT 
                    a.name area,
                    COUNT(v.id) cnt
                FROM vacancies v 
                JOIN areas a ON a.id = v.area_id
                GROUP BY a.id
                ORDER BY 2 DESC 
                LIMIT 15
            )

            SELECT *
            FROM x 

            UNION ALL

            SELECT         -- просуммируем число вакансий для топ 15 городов - total
                'total',
                sum(cnt)
            FROM x
            ORDER BY 2 DESC
            '''
            
df = pd.read_sql_query(query_4_6, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3902446055.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_6, connection)


,area,cnt
0,total,26154.0
1,Москва,5333.0
2,Санкт-Петербург,2851.0
3,Минск,2112.0
4,Новосибирск,2006.0
5,Алматы,1892.0
6,Екатеринбург,1698.0
7,Нижний Новгород,1670.0
8,Казань,1415.0
9,Краснодар,1301.0


***

#### Выводы по детальному анализу вакансий

По результатам запросов мы узнали, что:
* В топ 5 по количеству вакансий входят: Москва, Питер, Минск, Новосибирск и Алматы. По этой выборке видно, что количество вакансий прямо пропорционально населению города (-13, 5.3, 2, 1.6, 2.1 млн. чел соответственно), единственный город, который из закономерности выбивается это Алматы, скорее всего это связано, что в Казахстане есть аналогичный популярные площадки по размещению вакансий, конкурирующие с hh и оттягивающие часть работодателей и соискателей на себя.
* Только в 24 073 вакансий из 49 197 заполнено хотя бы одно из двух полей с зарплатой (меньше половины), хоть данная тенденция - не озвучивать уровень предлагаемой зарплаты, усложняет процесс выбора подходящих вакансий для соискателей, так как человек не может сразу, до собеседований, отбросить не устраивающие его варианты, это дает возможность работодателю более гибко подстраивать предложение по зарплате по итогам собеседования конкретному кандидату и не раскрывать публично уровень ЗП, который часто является коммерческой тайной. 
* Хотя в нашей выборке есть несколько городов из СНГ, для приблизительной оценки возьмем, что по данным РОССТАТА средняя зарплата в России за 2023 год ‒ 56 201 рублей, по результату же нашего запроса мы видим что средний показатель вилок - от 71065 до 110537 руб. Сделав дополнительный запрос 4_6, мы видим что на топ 15 городов по количеству вакансий приходится больше половины от общего числа, при том что по населению это меньше 20% (если брать отдельно только российские города), поэтому наше среднее значение сдвигается в большую сторону отражая больше уровень зарплат в городах миллионниках (а оно к примеру по данным за 22 год в Москве 128 т.руб, а в Питере 85 т.руб), чем в среднем по России. Это может быть связано с тем, что в городах миллионниках больше быстрорастущих компаний с большим числом вакансий, весь топ-менеджмент большинства крупных компаний сидит в основном в Москве, а также в них люди чаще меняют работу - больше вакансий с высоким уровнем ЗП двигает средний показатель ЗП вверх. Плюс в больших городах люди чаще используют интернет площадки для поиска вакансий, чем рост внутри компании или трудоустройство по знакомству. Также стоит отметить что обычно работодатели начинают чаще указывать примерный уровень ЗП на более высоких позициях (чтобы заинтересовать специалистов более высокого уровня), чем на начальных с низкой ЗП.
* По нашему запросу самое популярное сочетание типа рабочего графика - это "Полный день - Полная занятость" с количеством вакансий больше 35 000 (то есть больше 70%) - классический рабочий график, второе место, с 7802 вакансии занимает "Удаленная работа - Полная занятость" - не удивительно, учитывая что по прошлому блоку запросов мы выяснили, что из 5 компаний по числу вакансий 3 это IT и телеком компании - Яндекс, СБЕР и РТК.
  Самые непопулярные сочетания - это "Вахтовый метод - Проектная работа" и "Сменный график - Проектная работа" - достаточно редкие и узкопрофильные сочетания.
* Значения поля "Требуемый опыт работы (experience)" расположилось в порядке возрастания числа вакансий - "Более 6 лет — Нет опыта — От 3 до 6 лет — От 1 года до 3 лет", как правило в первые годы чаще всего наблюдается быстрый карьерный рост и смена мест работы - соответственно и больше текучка кадров  и освобождающихся вакансий - молодые люди начинают строить свою карьеру, после чего, с увеличением опыта люди как правило больше сидят на одном месте, так как, кто-то достигает желаемых сфер-должностей-заработных плат, а кто то смещает фокус на семью и личную жизнь. Также обычно вакансии на высокие должности занимают люли из этой же компании, и их не размещают на площадках в интернете. 
  Что же касается значения "Нет опыта", часто работодатели даже на базовых вакансиях указывают от 1 до 3 лет, а для людей совсем без опыта после учебы предлагают сначала программы стажировок.


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [89]:
# текст запроса

query_5_1= f'''
                WITH x AS (                 -- выделим отдельно сам запрос по ТОП работодателей по
                                            -- числу вакансий для улучшения читаемости кода

                    SELECT  
                        e.name employer,
                        COUNT(v.id)
                    FROM vacancies v
                    JOIN employers e ON v.employer_id = e.id
                    GROUP BY e.id
                    ORDER BY COUNT(v.id) DESC
                    )

            
                (
                SELECT * , '1 место'as position
                FROM x
                LIMIT 1
                )
                UNION  ALL 
                (
                SELECT * , '5 место'
                FROM x
                OFFSET 4 LIMIT 1
                )
            '''


In [90]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\1127888558.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


,employer,count,position
0,Яндекс,1933,1 место
1,Газпром нефть,331,5 место


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [91]:
# текст запроса
query_5_2= f'''
                SELECT            
                    region,
                    vacancies_cnt,
                    employers_cnt
                
                FROM (         -- сначала получим количество вакансий для каждого региона
                    SELECT 
                        a.id,
                        a.name region,
                        COUNT(v.id) vacancies_cnt
                    FROM areas a
                    LEFT JOIN vacancies v ON a.id = v.area_id
                    GROUP BY a.id
                    ) area_vacancies
                    
                    -- теперь соединим с количеством компаний зарегистрированных в конкретном регионе
                LEFT JOIN (  
                        SELECT 
                            area,
                            COUNT(id) employers_cnt
                        FROM employers
                        GROUP BY area) e 
                    ON e.area = area_vacancies.id
                WHERE vacancies_cnt = 0
                ORDER BY employers_cnt DESC NULLS LAST
                LIMIT 10
            '''

In [92]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\2002098862.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


,region,vacancies_cnt,employers_cnt
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
5,Ростовская область,0,18
6,Азербайджан,0,17
7,Нижегородская область,0,16
8,Республика Татарстан,0,16
9,Узбекистан,0,15


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [93]:
# текст запроса
query_5_3= f'''
                SELECT 
                    employer_id,
                    COUNT(DISTINCT area_id) vac_areas_cnt
                FROM vacancies v
                GROUP BY employer_id
                ORDER BY 2 DESC
            '''
            
# для ответа на вопрос "Выберите максимальное значение из получившегося списка." не требуется имя компании,
# если бы оно все таки было нужно запрос бы выглядил так:

query_5_3_additional= f'''
                            SELECT 
                                employer_id,
                                e.name,
                                COUNT(DISTINCT area_id) vac_areas_cnt
                            FROM vacancies v
                            JOIN employers e ON e.id = v.employer_id
                            GROUP BY v.employer_id, e.name
                            ORDER BY 3 DESC
                        '''

In [94]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3953667891.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


,employer_id,vac_areas_cnt
0,1740,181
1,2748,152
2,5724811,116
3,5130287,88
4,3682876,71
...,...,...
14901,810278,1
14902,810313,1
14903,810551,1
14904,810688,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [95]:
# текст запроса
query_5_4= f'''
                SELECT 
                    COUNT(e.id)
                FROM employers e 
                LEFT JOIN employers_industries e_i ON e_i.employer_id = e.id
                WHERE e_i.industry_id IS NULL
            '''

In [96]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\375529716.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [97]:
# текст запроса

query_5_5= f'''
                SELECT 
                    e.name
                FROM employers_industries e_i
                JOIN employers e ON e.id = e_i.employer_id
                GROUP BY e.id
                HAVING COUNT(e_i.industry_id) = 4 
                ORDER BY 1
                OFFSET 2 LIMIT 1
            '''

In [98]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3277383447.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [99]:
# текст запроса
query_5_6= f'''
                SELECT 
                    COUNT(DISTINCT e_i.employer_id)
                FROM employers_industries e_i
                JOIN industries i ON i.id = e_i.industry_id
                WHERE i.name = 'Разработка программного обеспечения'
            '''

In [100]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\1761488907.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 


In [101]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
header={'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=header)
page = BeautifulSoup(response.text, 'html.parser') 
cities = []
for child in page.find('table', class_='standard sortable').tbody:  
     # Перебираем имена ссылок в теле таблицы
     # в которых находятся названия городов.
     cities.append(child.a.text)
del cities[0]   # удаляем заголовок столбца с городами
cities = "'" + "', '".join(cities) + "'"  #преобразуем в формат для SQL запроса
cities

"'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград'"

In [102]:
# текст запроса


# удобно было бы для повторяющийся части запроса создать временную таблицу, но нет прав на запись(
query_5_7= f'''
                WITH id_ya AS (         -- найдем id яндекса в таблице работадатей
                    SELECT id 
                    FROM employers
                    WHERE name = 'Яндекс'
                )

                SELECT                  -- список регионов-миллионников, в которых представлены
                                        -- вакансии компании, вместе с количеством вакансий
                    a.name,
                    COUNT(distinct v.id) cnt 
                FROM vacancies v 
                JOIN areas a ON a.id = v.area_id
                WHERE a.name IN ({cities}) 
                    AND v.employer_id = (
                                        SELECT * 
                                        FROM id_ya
                                        )
                GROUP BY a.name


                UNION ALL 
                                      
                SELECT                  -- объединение со строчкой total суммирующей вакансии
                    'total',
                    SUM(cnt)
                FROM (SELECT 
                    a.name,
                    COUNT(distinct v.id) cnt 
                FROM vacancies v 
                JOIN areas a ON a.id = v.area_id
                WHERE a.name IN ({cities}) 
                    AND v.employer_id = (
                                        SELECT * 
                                        FROM id_ya
                                        )
                GROUP BY a.name) yandex
                ORDER BY 2
            '''


In [103]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\4127784153.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


---
#### Дополнительные запросы для дополнения выводов по блоку

запрос на подчет, какое количество работодателей разместили ровно 1 ваканчию

In [104]:
query_5_8 = f'''
                WITH x AS (                 -- переиспользуем уже написанный в задании 5_1 запрос
                    SELECT  
                        e.name employer ,
                        COUNT(v.id) cnt
                    FROM vacancies v
                    JOIN employers e ON v.employer_id = e.id
                    GROUP BY e.id
                    ORDER BY COUNT(v.id) DESC
                    )
                
                SELECT 
                    COUNT(*)
                FROM x
                WHERE cnt=1
            '''

df = pd.read_sql_query(query_5_8, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\923331115.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_8, connection)


,count
0,8699


Узнаем к запросу 5_3 дополнительно, название компаний и число регионов в которых они публикуют вакансии

In [105]:
query_5_3_additional= f'''
                            SELECT 
                                employer_id,
                                e.name,
                                COUNT(DISTINCT area_id) vac_areas_cnt
                            FROM vacancies v
                            JOIN employers e ON e.id = v.employer_id
                            GROUP BY v.employer_id, e.name
                            ORDER BY 3 DESC
                        '''

df = pd.read_sql_query(query_5_3_additional, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3553919113.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3_additional, connection)


,employer_id,name,vac_areas_cnt
0,1740,Яндекс,181
1,2748,Ростелеком,152
2,5724811,Спецремонт,116
3,5130287,Поляков Денис Иванович,88
4,3682876,ООО ЕФИН,71
...,...,...,...
14901,810278,НПП Авиатрон,1
14902,810313,Центр дистанционных торгов,1
14903,810551,Городские Телекоммуникационные Системы,1
14904,810688,"Введенский, Отель",1


Узнаем распределние количества компаний по признаку "Количество сфер деятельности"

In [106]:
query_5_9 = ''
n = 8
for i in range(1,n+1):
    query_5_9 += f'''
                    SELECT 
                        COUNT(name),
                        '{i}' N_industries
                    FROM(
                        SELECT 
                            e.name
                        FROM employers_industries e_i
                        JOIN employers e ON e.id = e_i.employer_id
                        GROUP BY e.id
                        HAVING COUNT(e_i.industry_id) = {i} 
                        ) x{i}
    '''
    if i<n:
        query_5_9+=" UNION ALL "
    else:
        query_5_9+="ORDER BY 1"
        
df = pd.read_sql_query(query_5_9, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3368861108.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_9, connection)


,count,n_industries
0,0,8
1,0,6
2,0,7
3,1138,4
4,1246,5
5,2607,3
6,3639,2
7,6452,1


Посмотрим на топ сфер деятельности по количеству работодателей в них

In [107]:
# текст запроса
query_5_10= f'''
                SELECT 
                    i.name,
                    COUNT(DISTINCT e_i.employer_id)
                FROM employers_industries e_i
                JOIN industries i ON i.id = e_i.industry_id
                GROUP BY i.id
                ORDER BY 2 DESC
            '''

# результат запроса
df = pd.read_sql_query(query_5_10, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\766467408.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_10, connection)


,name,count
0,Разработка программного обеспечения,3553
1,"Системная интеграция, автоматизации технологи...",2993
2,"Интернет-компания (поисковики, платежные систе...",1675
3,"Маркетинговые, рекламные, BTL, дизайнерские, E...",798
4,Консалтинговые услуги,662
...,...,...
289,Ритуальные услуги,4
290,Благотворительная организация,4
291,Автошкола,3
292,"Ботанический сад, зоопарк, заповедник",3


---

#### Выводы по детальному анализу работодателей

По результатам запросов мы узнали, что:
* 1 место по количеству вакансий занимает Яндекс, причем примерно в 4 раза обгоняя вторую компанию в ТОПе - Ростелеком. В результате запроса, если не ограничиваться первой и 5 позицией получаем примерно 14.9к работодателей, у которых размещена хотя бы одна вакансия. Также интересно отметить что медианное значения числа вакансий размещенных конкретным работодателем равно "1", причем, выполнив дополнительный запрос 5_8, получаем, что 8699 работодателей размещают ровно 1 вакансию.
* В результате запроса, мы получаем, что среди регионов, в которых нет вакансий, наибольшее количество работодателей в регионе "Россия", однако тут стоит обратить внимание, что под регионом в наших таблицах может быть представлен как город, так и вся область или страна, и на самом деле просто в таблице вакансий почти всегда используют город или иной населенный пункт, а в таблице работодателей достаточно часто встречается страна в качестве региона, поэтому мы и получаем в нашем запросе 5_2, что выборка регионов без вакансий в основном состоит их стран и областей (хотя и есть несколько маленьких населенных пунктов, где действительно нет вакансий).
* Больше всего (181) регион размещения вакансий у Яндекса, однако, если посмотреть не только топ 1, а весь результат запроса, видно что списки компаний по количеству регионов размещения вакансий и просто по количеству вакансий сильно различаются, и ТОП 5 компаний по количеству вакансий, который мы получили раньше, за исключением Яндекса, не входят в ТОП по регионам, что говорит о том, что скорее всего деятельность многих крупных компаний сконцентрирована в основном в нескольких крупных городах.
* У 8419 работодателей не указана сфера деятельности, учитывая что всего их в нашей таблице 23.5к, это достаточно большая доля пропущенных данных, что может повлиять на возможность использование данного признака для обучения модели.
* Сделав дополнительный запрос 5_9, мы видим, что максимальное число сфер деятельности у одной компании - 5, причем количество компаний обратно пропорционально (за исключением примерно одинакового числа для 4 и 5 сфер деятельности) числу указанных для компании сфер деятельности - то есть чаще всего компании специализируются на одной или двух сферах.
* По результату запроса 3553 работодателя указали одной из сфер деятельности «Разработка программного обеспечения». Если сделать дополнительный запрос 5_10, то окажется что это сфера заняла 1 место по количеству работодателей, а также что второе и третье место с сильным отрывом также занимают сферы IT - "Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг"  и "Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее)", что показывает бурный рост данный сферы.
* По полученному числу вакансий размещенных Яндексом в городах миллионниках, можно увидеть, что распределение достаточно равномерное, и хоть население Москвы  более чем в 10, а Санкт-Петербурга в 3 раза превышают население большинства городов из списка, количество вакансий  в них не настолько больше. Это может говорить о развитие компании офисов в других городах, например из-за меньшего уровня ЗП.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.
\


In [108]:

# текст запроса
query_6_1= f'''
             SELECT 
                COUNT(id)
            FROM vacancies
            WHERE name ILIKE '%data%'
                OR name ILIKE '%данн%'   
            '''

In [109]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\2931938894.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [110]:
# текст запроса
query_6_2= '''
                SELECT 
                    COUNT(*)
                FROM vacancies
                WHERE((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ) 
                     )
                     AND (name ILIKE '%junior%'         -- условия для начинающего специалиста
                        OR experience = 'Нет опыта'
                        OR employment = 'Стажировка'
                        )
                       
            '''

In [111]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\700150659.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [112]:
# текст запроса
query_6_3= '''
                SELECT 
                    COUNT(*)
                FROM vacancies
                WHERE((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ) 
                     )
                     AND key_skills ILIKE any ('{%SQL%, %postgres%}')
                         
            '''

In [113]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\1206811184.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.



In [114]:
# текст запроса
query_6_4= '''
                SELECT 
                    COUNT(*)
                FROM vacancies
                WHERE((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ) 
                     )
                     AND key_skills ILIKE '%python%'
            '''

In [115]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\4143830933.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [116]:
# текст запроса
query_6_5= '''
                SELECT 
                round(
                    AVG(
                        LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9),'')) + 1
                    )
                    ,2)
                FROM vacancies
                WHERE ((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ))
                    AND key_skills IS NOT NULL
                     
            '''

In [117]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3940953658.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [118]:
# текст запроса
query_6_6= '''
                SELECT
                experience,
                ROUND(
                AVG(COALESCE((salary_from+salary_to)/2, salary_to, salary_from)) 
                ) avg_salary
                FROM vacancies
                WHERE ((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ))
                    AND COALESCE(salary_from, salary_to) IS NOT NULL
                GROUP BY experience
                     
            '''

In [119]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3373444971.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


---
#### Дополнительные запросы для дополнения выводов по блоку

Проанализируем подробнее вакансии, которые имеют отношение к данным по сферам:


In [120]:
# текст запроса
query_6_7= '''
                SELECT 
                    COUNT(*),
                    'data science' sphere
                FROM vacancies
                WHERE((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ) 
                     )
                
                UNION ALL
                
                SELECT 
                    COUNT(*),
                    'аналитик данных'
                FROM vacancies
                WHERE name ILIKE any ('
                        { %аналитик%данн%, %data%analy% }
                        ')
        
                UNION ALL
                
                SELECT 
                    COUNT(*),
                    'администратор баз данных и хранилищ'
                FROM vacancies
                WHERE name ILIKE any ('
                        { %баз%данн%, %хранил%дан%, %Databas%Admin%, %админ%данн% }
                        ')

                UNION ALL
                
                SELECT 
                    COUNT(*),
                    'data engineer'
                FROM vacancies
                WHERE name ILIKE any ('
                        { %data%engineer%}
                        ')
                     
                ORDER BY 1
            '''
            
# результат запроса
df = pd.read_sql_query(query_6_7, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\2739303961.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_7, connection)


,count,sphere
0,315,data engineer
1,340,администратор баз данных и хранилищ
2,480,data science
3,502,аналитик данных


Оценим количество вакансий для DS в разрезе требуемого опыта работы

In [121]:
# текст запроса
query_6_8= '''
                SELECT 
                    COUNT(*),
                    experience
                FROM vacancies
                WHERE((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ) 
                     )
                GROUP BY experience
                ORDER BY 1
    
            '''
            
# результат запроса
df = pd.read_sql_query(query_6_8, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\472054411.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_8, connection)


,count,experience
0,28,Нет опыта
1,34,Более 6 лет
2,195,От 3 до 6 лет
3,223,От 1 года до 3 лет


Количество вакансий для начинающих дата-сайентистов, где указада хотябы 1 значения зарплатной вилки

In [122]:
# текст запроса
query_6_9= '''
                SELECT 
                    COUNT(*) salary_is_not_null,
                    'для начинающих' experience 
                FROM vacancies
                WHERE((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ) 
                     )
                     AND (name ILIKE '%junior%' 
                        OR experience = 'Нет опыта'
                        OR employment = 'Стажировка'
                        ) 
                    AND COALESCE(salary_from, salary_to) IS NOT NULL
                
                UNION ALL
                
                SELECT 
                    COUNT(*),
                    'всего'
                FROM vacancies
                WHERE((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ) 
                     )
                    AND COALESCE(salary_from, salary_to) IS NOT NULL
            '''
            
# результат запроса
df = pd.read_sql_query(query_6_9, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\2822015733.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_9, connection)


,salary_is_not_null,experience
0,62,всего
1,9,для начинающих


***

#### Выводы по предметному анализу

По результатам запросов мы узнали, что:
* 1771 вакансий имеет отношения к данным, что составляет примерно 3,5% от общего числа. Однако маска нашего поиска в первом запросе достаточно общая, так как слово "данные" встречается в IT достаточно часто, выполнив дополнительный запрос 6_7, оценим сколько вакансий по чаще всего встречающимся направлениям с ключевым словом "данн.." или "data". Получаем, что на первом месте вакансии связанные с аналитикой данных с 502 вакансиями, затем 480 вакансий для дата-сайентистов, после идут администраторы баз данных и дата инженеры.
* Для начинающих дата-сайентистов подходит 51 вакансия, что составляет примерно 10% от общего числа (480) вакансий в данной сфере. Однако сделав дополнительные запросы с маской по "middle" и "senior" мы получаем 44 и 63 вакансии соответственно, что покажет нам, что зачастую требуемый уровень работы просто не указан в названии вакансии и его оценка требует дополнительного анализа уже по атрибуту требуемого опыта. Сделав запрос 6_8 и оценив количество вакансий в разрезе требуемого опыта работы, можно сделать вывод, что более 85% вакансий в data science ориентированы на кандидатов с опытом работы от 1 до 6 лет, однако если учитывать, что большинство работодателей, даже на позицию "junior", указывают от 1 года опыта работы, можно сказать что и для начинающего дата-сайентиста найдутся вакансии).
* Среди вакансий для DS, в 201 был указан в качестве ключевого навыка SQL или Postgres, что говорит о важности умения работы с БД для дата-сайентиста.
* Аналогично важно знание Python, он был указан в навыках в более чем 73% вакансий (351 из 480)
* В среднем в вакансиях для DS указано 6.41 навыка, что практически равно среднему значению для всех вакансий в 6.37 навыков. Однако так как этот атрибут подразумевает именно ключевые навыки, а не вообще все, по нему нельзя сделать выводы например у каких сфер работы их требуется больше или меньше.
* Средний уровень ЗП для DS без опыта работы равен примерно 75к, от 1 до 3 лет 140к, от 3 до 6 - 243. Однако стоит отметить, что доп. запрос 6_9 показывает, что хотя бы одна граница зарплаты указана для 62 вакансий из 480 для DS вообще, и всего для 9 из 51 вакансии для начинающих дата-сайентистов, это достаточно малый процент для точной оценки.

# Общий вывод по проекту

#### подведем итог исследования, обобщите выводы

В результате проведенного анализа видно, что

* На топ 15 работодателей (запрос 7_2) приходится более 11% от всех вакансий, причем 13 из 15 компаний, так или иначе связаны со сферой IT (либо предоставляют услуги в IT, либо имеют большие IT команды (К примеру Тинькофф или СБЕР)).
  
  В блоке по детальному анализу вакансий я уже подробно описывал какие чаще всего значения атрибутов в вакансиях встречаются и с чем это может быть связано. Подводя итоги, можно еще раз отметить, что самое популярное сочетание типа рабочего графика - это "Полный день - Полная занятость" с количеством вакансий больше 35 000 (то есть больше 70%) - классический рабочий график, второе место, с 7802 вакансии занимает "Удаленная работа - Полная занятость" - данный график стал очень популярен у людей после эпидемии, так как он позволяет сочетать работу и личные дела, а также не тратить время на дорогу, а у многих компаний теперь появились технические возможности, для данного типа работы. Чаще всего значение "Требуемый опыт работы (experience)" равно От 1 года до 3 лет.

* Можно отметить, что больше 40% вакансий приходится на топ 10 городов (доп запрос 7_1). Если посмотреть на количество работодателей в этих городах, то первое место занимает Москва с отрывом от остальных регионов России более чем в 10 раз, так как многие крупные компании, даже те, у кого много сотрудников и представительств в регионах, например горно- и нефтедобывающие, все равно размещают главный офис(и юр. лицо) в Москве. Однако если сравнивать именно количество вакансий, то в Москве их всего в 3-4 раза больше, чем в других городах миллионниках России(менее чем в 2 раза больше чем в Питере), что при разницы населения в среднем в 10 раз, может говорить о тенденциях децентрализации, и открытия все большего количества офисов и предприятий по всей территории России. Причинами может являться более низкая стоимость земли/аренды так и меньший уровень ЗП, а с учетом высокоскоростных сетей и современных технологий, офисы компании, расположенные по всей стране, могут функционировать как единое целое, а в IT компаниях, например в ранее рассматриваемом Яндексе, такая тенденция становится еще заметнее.

* В качестве еще дополнительных исследований, давайте выполним запрос 7_3 (заодно попробуем использовать POSIX регулярные выражения) и сравним насколько востребованы разработчики без опыта на языках JAVA и Python. По результатам видно, что количество вакансий для стажеров и junior разработчиков на Java более чем в 2 раза превышает их для Python (108 против 48). Однако, во-первых, как мы ранее выяснили, знание (и соответственно использование)  Python,очень часто требуется например в data science, а эти вакансии мы не учитывали в нашем запросе, а во-вторых, если судить по первому проекту, данные от HH даны за 2019 - 2020 год, а Python сейчас с каждым годом увеличивает свою популярность, и например по данным за 2022 год популярных зарубежных сайтов по поиску вакансий IEEE и CareerBuilder, Python практически сравнялся с Java - 88 против 95 условных баллов.

* Давайте также проанализируем зависимость ЗП от типа работы и занятости в DS. По результату запроса 7_4 логично получаем, что самый низкий уровень у стажировок, потом идут частичная занятость, за ней полная, однако, интересно то, что и среди вакансий на частичную занятость и на полную, выше уровень зарплат у удаленной работы, за которой идет классический полный день и на последнем месте гибкий график. Дополнительно добавим в наш запрос группировку еще и по опыту работы. Получаем что для опыта работы от 3 до 6 лет (можем соотносить это с уровнем senior) при классическом полном дне или гибком графике уровень ЗП выше чем при удаленной работе, а вот для вакансий с требуемым опытом от 1 до 3 лет, ситуация обратная - при удаленной работе ЗП существенно выше. Однако, к сожалению, как я отметил ранее, во многих вакансиях уровень ЗП не указан (только 62 вакансии из 480 имеют хотя бы один заполненный атрибут из 2х), поэтому по такой небольшой выборке трудно сказать, что наши выводы точно применимы к реальным раскладам по рынку ЗП в DS.










#### Здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования


Оценим ТОП 10 регионов по числу вакансий, дополнительно выводя количество работодателей зарегистрированных в этих регионах.

In [123]:
# текст запроса
query_7_1= '''
                SELECT 
                    region,
                    vacancies_cnt,
                    employers_cnt
                    
                FROM ( 
                    SELECT 
                        a.id,
                        a.name region,
                        COUNT(v.id) vacancies_cnt
                    FROM areas a
                    LEFT JOIN vacancies v ON a.id = v.area_id
                    GROUP BY a.id
                    ) area_vacancies

                LEFT JOIN (
                        SELECT 
                            area,
                            COUNT(id) employers_cnt
                        FROM employers
                        GROUP BY area) e 
                    ON e.area = area_vacancies.id
               
                ORDER BY vacancies_cnt DESC NULLS LAST
                LIMIT 10
    
            '''
            
# результат запроса
df = pd.read_sql_query(query_7_1, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\638207653.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_7_1, connection)


,region,vacancies_cnt,employers_cnt
0,Москва,5333,5864
1,Санкт-Петербург,2851,2217
2,Минск,2112,1115
3,Новосибирск,2006,573
4,Алматы,1892,721
5,Екатеринбург,1698,609
6,Нижний Новгород,1670,426
7,Казань,1415,480
8,Краснодар,1301,409
9,Самара,1144,315


Топ 15 компаний по числу вакансий.

In [124]:
# текст запроса
query_7_2= '''
                (SELECT  
                    e.name employer,
                    COUNT(v.id) cnt
                FROM vacancies v
                JOIN employers e ON v.employer_id = e.id
                GROUP BY e.id
                ORDER BY COUNT(v.id) DESC
                LIMIT 15)
                
                UNION ALL
                
                SELECT 
                    'total',
                    SUM(cnt)
                FROM (
                    SELECT  
                        e.name employer,
                        COUNT(v.id) cnt
                    FROM vacancies v
                    JOIN employers e ON v.employer_id = e.id
                    GROUP BY e.id
                    ORDER BY COUNT(v.id) DESC
                    LIMIT 15
                ) x
                
    
            '''
            
# результат запроса
df = pd.read_sql_query(query_7_2, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\195019184.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_7_2, connection)


,employer,cnt
0,Яндекс,1933.0
1,Ростелеком,491.0
2,Тинькофф,444.0
3,СБЕР,428.0
4,Газпром нефть,331.0
5,ИК СИБИНТЕК,327.0
6,МТС,292.0
7,DataArt,247.0
8,Совкомбанк Технологии,204.0
9,Первый Бит,176.0


Сравнение количества вакансий для разработчиков без опыта на Java или Python.

In [125]:
# текст запроса
query_7_3= '''
                SELECT 
                    COUNT(*),
                    'вакансии для начинающего разработчика на Python' "language"
                FROM vacancies
                WHERE(name ~*'.*разработ.*python.*|.*python.*разработ.*|.*Python.*Developer.*|.*developer.*python.*|.*программист.*python.*'
                       
                     )
                     AND (name ~* '.*junior.*'       -- условия для начинающего специалиста
                        OR experience = 'Нет опыта'
                        OR employment = 'Стажировка'
                        )
                        
                UNION ALL
                
                SELECT 
                    COUNT(*),
                    'вакансии для начинающего разработчика на JAVA'
                FROM vacancies
                WHERE name ~*'.*разработ.*Java.*|.*Java.*разработ.*|.*Java.*Developer.*|.*developer.*Java.*|.*программист.*Java.*'
                 
                    AND name !~* '.*Java.*Script.*'  -- Но нас не интересует JavaScript
    
                    AND (name ~* '.*junior.*' 
                        OR experience = 'Нет опыта'
                        OR employment = 'Стажировка'
                        )
            '''
            
# результат запроса
df = pd.read_sql_query(query_7_3, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3801018459.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_7_3, connection)


,count,language
0,48,вакансии для начинающего разработчика на Python
1,108,вакансии для начинающего разработчика на JAVA


Зависимость ЗП от типа работы и занятости в DS

In [126]:
# текст запроса
query_7_4= '''
                SELECT
                schedule || ' - ' || employment combination,
                ROUND(
                AVG(COALESCE((salary_from+salary_to)/2, salary_to, salary_from)) 
                ) avg_salary
                FROM vacancies
                WHERE ((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ))
                    AND COALESCE(salary_from, salary_to) IS NOT NULL
                GROUP BY schedule || ' - ' || employment
                ORDER BY 2
            '''
            
# результат запроса
df = pd.read_sql_query(query_7_4, connection)
display(df)

# текст запроса
query_7_4_2= '''
                SELECT
                experience,
                schedule || ' - ' || employment combination,
                ROUND(
                AVG(COALESCE((salary_from+salary_to)/2, salary_to, salary_from)) 
                ) avg_salary
                FROM vacancies
                WHERE ((name ILIKE any ('
                        { %data scientist%, %data science%, %исследователь данных%,
                        %machine learning%, %машинн%обучен% }
                        '))
                        OR  (
                            name LIKE '%ML%' 
                            AND name not LIKE '%HTML%'
                        ))
                    AND COALESCE(salary_from, salary_to) IS NOT NULL
                GROUP BY experience, schedule || ' - ' || employment 
                ORDER BY 3
            '''
            
# результат запроса
df = pd.read_sql_query(query_7_4_2, connection)
display(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3407180591.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_7_4, connection)


,combination,avg_salary
0,Гибкий график - Стажировка,37000.0
1,Полный день - Частичная занятость,72500.0
2,Удаленная работа - Проектная работа,90000.0
3,Удаленная работа - Частичная занятость,119000.0
4,Гибкий график - Полная занятость,175000.0
5,Полный день - Полная занятость,178307.0
6,Удаленная работа - Полная занятость,209409.0


C:\Users\maxim\AppData\Local\Temp\ipykernel_1856\3407180591.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_7_4_2, connection)


,experience,combination,avg_salary
0,Нет опыта,Гибкий график - Стажировка,37000.0
1,Нет опыта,Полный день - Частичная занятость,72500.0
2,Нет опыта,Полный день - Полная занятость,82600.0
3,От 1 года до 3 лет,Удаленная работа - Проектная работа,90000.0
4,От 3 до 6 лет,Удаленная работа - Частичная занятость,119000.0
5,От 1 года до 3 лет,Полный день - Полная занятость,132253.0
6,От 1 года до 3 лет,Гибкий график - Полная занятость,137500.0
7,От 1 года до 3 лет,Удаленная работа - Полная занятость,179224.0
8,От 3 до 6 лет,Удаленная работа - Полная занятость,232048.0
9,От 3 до 6 лет,Гибкий график - Полная занятость,250000.0
